### Deck simulator

This code is used to calculate rough estimates (approx. within 1 percent) of probabilities related to drawing cards, with an interest in reaching a specific type of hand, by simulating the event several thousand times and adding the results.

This is helpful in a game design context, when using special rules, in order to better understand different types of hands in terms of them having a certain probability of occurring.

One could certainly calculate the exact probabilities by devising the proper formula, but I thought this method would be less prone to reasoning errors and yielded precise enough answers to be considered useful.

In [1]:
import random

# Start by defining a card
class Card(object):
    def __init__(self, rank, suit):
        self.rank = rank
        self.suit = suit
    
    def sameSuit(self, other):
        return self.suit == other.suit
    
    def __eq__(self, other):
        return self.rank == other.rank and self.sameSuit(other)
    
    def __ge__(self, other):
        return self.rank >= other.rank
    
    def __gt__(self, other):
        return self.rank > other.rank
    
    def show(self):
        print("{}{}".format(self.rank, self.suit))

In [2]:
# Also need a deck for the collection of cards
class Deck(object):
    def __init__(self):
        self.cards = []
        self.fill()
        
    def fill(self):
        for s in ['S', 'C', 'H', 'D']:
            for r in range(1, 14):
                self.cards.append(Card(r, s))
    
    def clear(self):
        self.cards = []
    
    def show(self):
        for c in self.cards:
            c.show()
    
    def shuffle(self):
        random.shuffle(self.cards)
    
    def pickCard(self, target):
        if self.cards == []:
            return Exception("Cannot pick a card from an empty deck.")
        else:
            for c in self.cards:
                if c == target:
                    return self.cards.pop(c)
        return Exception("Cannot find {}{} in the deck".format(rank, suit))
    

In [3]:
# As well as the hand
class Hand(object):
    def __init__(self):
        self.cards = []
    
    def draw(self, deck, nof_cards=1):
        if nof_cards < 1:
            raise Exception("Must select a positive number amount of cards, not {}.".format(nof_cards)) 
        elif nof_cards > len(deck.cards):
            raise Exception("Not enough cards in the deck in order to draw {} cards.".format(nof_cards))
        else:
            for n in range(nof_cards):
                self.cards.append(deck.cards.pop(0))
    
    def show(self):
        for c in self.cards:
            c.show()
    
    def sort(self):
        self.cards.sort(key = lambda c: c.rank)    
    
    def returnTo(self, deck):
        while len(self.cards) != 0:
            deck.cards.append(self.cards.pop(0))
    
    def addCard(self, card):
        self.cards.append(card)
    
    def hasMatch(self, target):
        if self.cards == []:
            return False
        else:
            for c in self.cards:
                if c.rank >= target.rank and c.suit == target.suit:
                    return True
            return False
    
    def selectCard(self, target):
        if self.cards == []:
            return Exception("Cannot pick a card from an empty hand.")
        else:
            for c in self.cards:
                if c.sameSuit(target) and c >= target:
                    self.cards.remove(c)
                    return c
            return Exception("Cannot find {}{} in the deck".format(rank, suit))
    
    def checkSuccess(self, targetCards):
        tmp = self.cards.copy()
        for c in targetCards:
            if self.hasMatch(c):
                self.selectCard(c)
            else:
                self.cards = tmp
                return False
        self.cards = tmp
        return True

In [4]:
# Define proper test for success
def testProbability(deck, hand, hand_size, trial_name, trial_cards, nof_tries):
    nof_successes = 0
    for n in range(nof_tries):
        hand.returnTo(deck)
        deck.shuffle()
        hand.draw(deck, hand_size)
        hand.sort()
        nof_successes += hand.checkSuccess(trial_cards)
    
    success_rate = 100 * nof_successes / nof_tries
    cards = ""
    for c in trial_cards:
        cards+=str(c.rank)
        cards+=str(c.suit)
        cards+=" "
        
    print(trial_name, cards, "Using hand size of {0}, with {1} trials, has a success percentage of {2:.1f}".format(hand_size, nof_tries, success_rate)) 

In [5]:
deck = Deck()
hand = Hand()

In [6]:
# Example uses (in a tabletop gaming context)

trials = {
    "Alter Emotions": [Card(4, 'C'), Card(2, 'C')],
    "Alter Lighting": [Card(5, 'D'), Card(2, 'D')],
    "Armor Skin": [Card(4, 'D'), Card(1, 'D'), Card(2, 'H')],
    "Animal Attunement": [Card(6, 'C'), Card(3, 'C'), Card(3, 'H')],
    "Animal Companion": [Card(11, 'H'), Card(5, 'C')],
    "Bury": [Card(5, 'D'), Card(3, 'D'), Card(1, 'S')],
    "Crashing Wave": [Card(4, 'H'), Card(2, 'H'), Card(2, 'S'), Card(1, 'S')],
    "Cure": [Card(5, 'S'), Card(5, 'C')],
    "Drain": [Card(7, 'S'), Card(1, 'S')],
    "Dull Sense": [Card(6, 'S'), Card(3, 'C')],
    "Energy Ray": [Card(5, 'S'), Card(4, 'S'), Card(2, 'H')],
    "Energy Shield": [Card(6, 'D'), Card(1, 'S')],
    "Magical Attunement": [Card(7, 'C')],
    "Magical Rope": [Card(5, 'H'), Card(3, 'C'), Card(1, 'D')],
    "Poison Spray": [Card(3, 'H'), Card(2, 'S'), Card(1, 'S')],
    "Restoration": [Card(5, 'D'), Card(5, 'C')],
    "Ray of Disintegration": [Card(13, 'S')]
}

# for key, value in trials.items():
#     testProbability(deck, hand, 5, key, value, 100000)
#     testProbability(deck, hand, 10, key, value, 100000)

In [7]:
# Finding out the probabilities for each meaningful pair of cards

abstract_trials = {
    "Single 1": [Card(1, 'H')],
    "Single 2": [Card(2, 'H')],
    "Single 3": [Card(3, 'H')],
    "Single 4": [Card(4, 'H')],
    "Single 5": [Card(5, 'H')],
    "Single 6": [Card(6, 'H')],
    "Single 7": [Card(7, 'H')],
    "Single 8": [Card(8, 'H')],
    "Single 9": [Card(9, 'H')],
    "Single 10": [Card(10, 'H')],
    "Single 11": [Card(11, 'H')],
    "Single 12": [Card(12, 'H')],
    "Single 13": [Card(13, 'H')],
    "Pair 1": [Card(1, 'H'), Card(1, 'D')],
    "Pair 2": [Card(2, 'H'), Card(2, 'D')],
    "Pair 3": [Card(3, 'H'), Card(3, 'D')],
    "Pair 4": [Card(4, 'H'), Card(4, 'D')],
    "Pair 5": [Card(5, 'H'), Card(5, 'D')],
    "Pair 6": [Card(6, 'H'), Card(6, 'D')],
    "Pair 7": [Card(7, 'H'), Card(7, 'D')],
    "Pair 8": [Card(8, 'H'), Card(8, 'D')],
    "Pair 9": [Card(9, 'H'), Card(9, 'D')],
    "Pair 10": [Card(10, 'H'), Card(10, 'D')],
    "Pair 11": [Card(11, 'H'), Card(11, 'D')],
    "Pair 12": [Card(12, 'H'), Card(12, 'D')],
    "Pair 13": [Card(13, 'H'), Card(13, 'D')],
    "Suit 1 2": [Card(1, 'H'), Card(2, 'H')],
    "Suit 1 3": [Card(1, 'H'), Card(3, 'H')],
    "Suit 1 4": [Card(1, 'H'), Card(4, 'H')],
    "Suit 1 5": [Card(1, 'H'), Card(5, 'H')],
    "Suit 1 6": [Card(1, 'H'), Card(6, 'H')],
    "Suit 1 7": [Card(1, 'H'), Card(7, 'H')],
    "Suit 1 8": [Card(1, 'H'), Card(8, 'H')],
    "Suit 1 9": [Card(1, 'H'), Card(9, 'H')],
    "Suit 1 10": [Card(1, 'H'), Card(10, 'H')],
    "Suit 1 11": [Card(1, 'H'), Card(11, 'H')],
    "Suit 1 12": [Card(1, 'H'), Card(12, 'H')],
    "Suit 1 13": [Card(1, 'H'), Card(13, 'H')],
    "Suit 2 3": [Card(2, 'H'), Card(3, 'H')],
    "Suit 2 4": [Card(2, 'H'), Card(4, 'H')],
    "Suit 2 5": [Card(2, 'H'), Card(5, 'H')],
    "Suit 2 6": [Card(2, 'H'), Card(6, 'H')],
    "Suit 2 7": [Card(2, 'H'), Card(7, 'H')],
    "Suit 2 8": [Card(2, 'H'), Card(8, 'H')],
    "Suit 2 9": [Card(2, 'H'), Card(9, 'H')],
    "Suit 2 10": [Card(2, 'H'), Card(10, 'H')],
    "Suit 2 11": [Card(2, 'H'), Card(11, 'H')],
    "Suit 2 12": [Card(2, 'H'), Card(12, 'H')],
    "Suit 2 13": [Card(2, 'H'), Card(13, 'H')],
    "Suit 3 4": [Card(3, 'H'), Card(4, 'H')],
    "Suit 3 5": [Card(3, 'H'), Card(5, 'H')],
    "Suit 3 6": [Card(3, 'H'), Card(6, 'H')],
    "Suit 3 7": [Card(3, 'H'), Card(7, 'H')],
    "Suit 3 8": [Card(3, 'H'), Card(8, 'H')],
    "Suit 3 9": [Card(3, 'H'), Card(9, 'H')],
    "Suit 3 10": [Card(3, 'H'), Card(10, 'H')],
    "Suit 3 11": [Card(3, 'H'), Card(11, 'H')],
    "Suit 3 12": [Card(3, 'H'), Card(12, 'H')],
    "Suit 3 13": [Card(3, 'H'), Card(13, 'H')],
    "Suit 4 5": [Card(4, 'H'), Card(5, 'H')],
    "Suit 4 6": [Card(4, 'H'), Card(6, 'H')],
    "Suit 4 7": [Card(4, 'H'), Card(7, 'H')],
    "Suit 4 8": [Card(4, 'H'), Card(8, 'H')],
    "Suit 4 9": [Card(4, 'H'), Card(9, 'H')],
    "Suit 4 10": [Card(4, 'H'), Card(10, 'H')],
    "Suit 4 11": [Card(4, 'H'), Card(11, 'H')],
    "Suit 4 12": [Card(4, 'H'), Card(12, 'H')],
    "Suit 4 13": [Card(4, 'H'), Card(13, 'H')],
    "Suit 5 6": [Card(5, 'H'), Card(6, 'H')],
    "Suit 5 7": [Card(5, 'H'), Card(7, 'H')],
    "Suit 5 8": [Card(5, 'H'), Card(8, 'H')],
    "Suit 5 9": [Card(5, 'H'), Card(9, 'H')],
    "Suit 5 10": [Card(5, 'H'), Card(10, 'H')],
    "Suit 5 11": [Card(5, 'H'), Card(11, 'H')],
    "Suit 5 12": [Card(5, 'H'), Card(12, 'H')],
    "Suit 5 13": [Card(5, 'H'), Card(13, 'H')],
    "Suit 6 7": [Card(6, 'H'), Card(7, 'H')],
    "Suit 6 8": [Card(6, 'H'), Card(8, 'H')],
    "Suit 6 9": [Card(6, 'H'), Card(9, 'H')],
    "Suit 6 10": [Card(6, 'H'), Card(10, 'H')],
    "Suit 6 11": [Card(6, 'H'), Card(11, 'H')],
    "Suit 6 12": [Card(6, 'H'), Card(12, 'H')],
    "Suit 6 13": [Card(6, 'H'), Card(13, 'H')],
    "Suit 7 8": [Card(7, 'H'), Card(8, 'H')],
    "Suit 7 9": [Card(7, 'H'), Card(9, 'H')],
    "Suit 7 10": [Card(7, 'H'), Card(10, 'H')],
    "Suit 7 11": [Card(7, 'H'), Card(11, 'H')],
    "Suit 7 12": [Card(7, 'H'), Card(12, 'H')],
    "Suit 7 13": [Card(7, 'H'), Card(13, 'H')],
    "Suit 8 9": [Card(8, 'H'), Card(9, 'H')],
    "Suit 8 10": [Card(8, 'H'), Card(10, 'H')],
    "Suit 8 11": [Card(8, 'H'), Card(11, 'H')],
    "Suit 8 12": [Card(8, 'H'), Card(12, 'H')],
    "Suit 8 13": [Card(8, 'H'), Card(13, 'H')],
    "Suit 9 10": [Card(9, 'H'), Card(10, 'H')],
    "Suit 9 11": [Card(9, 'H'), Card(11, 'H')],
    "Suit 9 12": [Card(9, 'H'), Card(12, 'H')],
    "Suit 9 13": [Card(9, 'H'), Card(13, 'H')],
    "Suit 10 11": [Card(10, 'H'), Card(11, 'H')],
    "Suit 10 12": [Card(10, 'H'), Card(12, 'H')],
    "Suit 10 13": [Card(10, 'H'), Card(13, 'H')],
    "Suit 11 12": [Card(11, 'H'), Card(12, 'H')],
    "Suit 11 13": [Card(11, 'H'), Card(13, 'H')],
    "Suit 12 13": [Card(12, 'H'), Card(13, 'H')],
    "Offsuit 1 2": [Card(1, 'H'), Card(2, 'D')],
    "Offsuit 1 3": [Card(1, 'H'), Card(3, 'D')],
    "Offsuit 1 4": [Card(1, 'H'), Card(4, 'D')],
    "Offsuit 1 5": [Card(1, 'H'), Card(5, 'D')],
    "Offsuit 1 6": [Card(1, 'H'), Card(6, 'D')],
    "Offsuit 1 7": [Card(1, 'H'), Card(7, 'D')],
    "Offsuit 1 8": [Card(1, 'H'), Card(8, 'D')],
    "Offsuit 1 9": [Card(1, 'H'), Card(9, 'D')],
    "Offsuit 1 10": [Card(1, 'H'), Card(10, 'D')],
    "Offsuit 1 11": [Card(1, 'H'), Card(11, 'D')],
    "Offsuit 1 12": [Card(1, 'H'), Card(12, 'D')],
    "Offsuit 1 13": [Card(1, 'H'), Card(13, 'D')],
    "Offsuit 2 3": [Card(2, 'H'), Card(3, 'D')],
    "Offsuit 2 4": [Card(2, 'H'), Card(4, 'D')],
    "Offsuit 2 5": [Card(2, 'H'), Card(5, 'D')],
    "Offsuit 2 6": [Card(2, 'H'), Card(6, 'D')],
    "Offsuit 2 7": [Card(2, 'H'), Card(7, 'D')],
    "Offsuit 2 8": [Card(2, 'H'), Card(8, 'D')],
    "Offsuit 2 9": [Card(2, 'H'), Card(9, 'D')],
    "Offsuit 2 10": [Card(2, 'H'), Card(10, 'D')],
    "Offsuit 2 11": [Card(2, 'H'), Card(11, 'D')],
    "Offsuit 2 12": [Card(2, 'H'), Card(12, 'D')],
    "Offsuit 2 13": [Card(2, 'H'), Card(13, 'D')],
    "Offsuit 3 4": [Card(3, 'H'), Card(4, 'D')],
    "Offsuit 3 5": [Card(3, 'H'), Card(5, 'D')],
    "Offsuit 3 6": [Card(3, 'H'), Card(6, 'D')],
    "Offsuit 3 7": [Card(3, 'H'), Card(7, 'D')],
    "Offsuit 3 8": [Card(3, 'H'), Card(8, 'D')],
    "Offsuit 3 9": [Card(3, 'H'), Card(9, 'D')],
    "Offsuit 3 10": [Card(3, 'H'), Card(10, 'D')],
    "Offsuit 3 11": [Card(3, 'H'), Card(11, 'D')],
    "Offsuit 3 12": [Card(3, 'H'), Card(12, 'D')],
    "Offsuit 3 13": [Card(3, 'H'), Card(13, 'D')],
    "Offsuit 4 5": [Card(4, 'H'), Card(5, 'D')],
    "Offsuit 4 6": [Card(4, 'H'), Card(6, 'D')],
    "Offsuit 4 7": [Card(4, 'H'), Card(7, 'D')],
    "Offsuit 4 8": [Card(4, 'H'), Card(8, 'D')],
    "Offsuit 4 9": [Card(4, 'H'), Card(9, 'D')],
    "Offsuit 4 10": [Card(4, 'H'), Card(10, 'D')],
    "Offsuit 4 11": [Card(4, 'H'), Card(11, 'D')],
    "Offsuit 4 12": [Card(4, 'H'), Card(12, 'D')],
    "Offsuit 4 13": [Card(4, 'H'), Card(13, 'D')],
    "Offsuit 5 6": [Card(5, 'H'), Card(6, 'D')],
    "Offsuit 5 7": [Card(5, 'H'), Card(7, 'D')],
    "Offsuit 5 8": [Card(5, 'H'), Card(8, 'D')],
    "Offsuit 5 9": [Card(5, 'H'), Card(9, 'D')],
    "Offsuit 5 10": [Card(5, 'H'), Card(10, 'D')],
    "Offsuit 5 11": [Card(5, 'H'), Card(11, 'D')],
    "Offsuit 5 12": [Card(5, 'H'), Card(12, 'D')],
    "Offsuit 5 13": [Card(5, 'H'), Card(13, 'D')],
    "Offsuit 6 7": [Card(6, 'H'), Card(7, 'D')],
    "Offsuit 6 8": [Card(6, 'H'), Card(8, 'D')],
    "Offsuit 6 9": [Card(6, 'H'), Card(9, 'D')],
    "Offsuit 6 10": [Card(6, 'H'), Card(10, 'D')],
    "Offsuit 6 11": [Card(6, 'H'), Card(11, 'D')],
    "Offsuit 6 12": [Card(6, 'H'), Card(12, 'D')],
    "Offsuit 6 13": [Card(6, 'H'), Card(13, 'D')],
    "Offsuit 7 8": [Card(7, 'H'), Card(8, 'D')],
    "Offsuit 7 9": [Card(7, 'H'), Card(9, 'D')],
    "Offsuit 7 10": [Card(7, 'H'), Card(10, 'D')],
    "Offsuit 7 11": [Card(7, 'H'), Card(11, 'D')],
    "Offsuit 7 12": [Card(7, 'H'), Card(12, 'D')],
    "Offsuit 7 13": [Card(7, 'H'), Card(13, 'D')],
    "Offsuit 8 9": [Card(8, 'H'), Card(9, 'D')],
    "Offsuit 8 10": [Card(8, 'H'), Card(10, 'D')],
    "Offsuit 8 11": [Card(8, 'H'), Card(11, 'D')],
    "Offsuit 8 12": [Card(8, 'H'), Card(12, 'D')],
    "Offsuit 8 13": [Card(8, 'H'), Card(13, 'D')],
    "Offsuit 9 10": [Card(9, 'H'), Card(10, 'D')],
    "Offsuit 9 11": [Card(9, 'H'), Card(11, 'D')],
    "Offsuit 9 12": [Card(9, 'H'), Card(12, 'D')],
    "Offsuit 9 13": [Card(9, 'H'), Card(13, 'D')],
    "Offsuit 10 11": [Card(10, 'H'), Card(11, 'D')],
    "Offsuit 10 12": [Card(10, 'H'), Card(12, 'D')],
    "Offsuit 10 13": [Card(10, 'H'), Card(13, 'D')],
    "Offsuit 11 12": [Card(11, 'H'), Card(12, 'D')],
    "Offsuit 11 13": [Card(11, 'H'), Card(13, 'D')],
    "Offsuit 12 13": [Card(12, 'H'), Card(13, 'D')]
}

for key, value in abstract_trials.items():
    testProbability(deck, hand, 5, key, value, 100000)
    testProbability(deck, hand, 10, key, value, 100000)

Single 1 1H  Using hand size of 5, with 100000 trials, has a success percentage of 77.9
Single 1 1H  Using hand size of 10, with 100000 trials, has a success percentage of 95.9
Single 2 2H  Using hand size of 5, with 100000 trials, has a success percentage of 74.8
Single 2 2H  Using hand size of 10, with 100000 trials, has a success percentage of 94.6
Single 3 3H  Using hand size of 5, with 100000 trials, has a success percentage of 71.1
Single 3 3H  Using hand size of 10, with 100000 trials, has a success percentage of 92.9
Single 4 4H  Using hand size of 5, with 100000 trials, has a success percentage of 67.2
Single 4 4H  Using hand size of 10, with 100000 trials, has a success percentage of 90.6
Single 5 5H  Using hand size of 5, with 100000 trials, has a success percentage of 62.9
Single 5 5H  Using hand size of 10, with 100000 trials, has a success percentage of 87.8
Single 6 6H  Using hand size of 5, with 100000 trials, has a success percentage of 58.6
Single 6 6H  Using hand siz

Suit 2 10 2H 10H  Using hand size of 10, with 100000 trials, has a success percentage of 51.8
Suit 2 11 2H 11H  Using hand size of 5, with 100000 trials, has a success percentage of 16.2
Suit 2 11 2H 11H  Using hand size of 10, with 100000 trials, has a success percentage of 43.0
Suit 2 12 2H 12H  Using hand size of 5, with 100000 trials, has a success percentage of 11.4
Suit 2 12 2H 12H  Using hand size of 10, with 100000 trials, has a success percentage of 31.9
Suit 2 13 2H 13H  Using hand size of 5, with 100000 trials, has a success percentage of 6.1
Suit 2 13 2H 13H  Using hand size of 10, with 100000 trials, has a success percentage of 17.4
Suit 3 4 3H 4H  Using hand size of 5, with 100000 trials, has a success percentage of 28.1
Suit 3 4 3H 4H  Using hand size of 10, with 100000 trials, has a success percentage of 68.4
Suit 3 5 3H 5H  Using hand size of 5, with 100000 trials, has a success percentage of 27.7
Suit 3 5 3H 5H  Using hand size of 10, with 100000 trials, has a success

Suit 8 10 8H 10H  Using hand size of 5, with 100000 trials, has a success percentage of 8.9
Suit 8 10 8H 10H  Using hand size of 10, with 100000 trials, has a success percentage of 30.8
Suit 8 11 8H 11H  Using hand size of 5, with 100000 trials, has a success percentage of 7.6
Suit 8 11 8H 11H  Using hand size of 10, with 100000 trials, has a success percentage of 27.2
Suit 8 12 8H 12H  Using hand size of 5, with 100000 trials, has a success percentage of 6.0
Suit 8 12 8H 12H  Using hand size of 10, with 100000 trials, has a success percentage of 21.1
Suit 8 13 8H 13H  Using hand size of 5, with 100000 trials, has a success percentage of 3.3
Suit 8 13 8H 13H  Using hand size of 10, with 100000 trials, has a success percentage of 12.3
Suit 9 10 9H 10H  Using hand size of 5, with 100000 trials, has a success percentage of 6.6
Suit 9 10 9H 10H  Using hand size of 10, with 100000 trials, has a success percentage of 24.4
Suit 9 11 9H 11H  Using hand size of 5, with 100000 trials, has a succ

Offsuit 3 10 3H 10D  Using hand size of 10, with 100000 trials, has a success percentage of 53.7
Offsuit 3 11 3H 11D  Using hand size of 5, with 100000 trials, has a success percentage of 17.1
Offsuit 3 11 3H 11D  Using hand size of 10, with 100000 trials, has a success percentage of 43.8
Offsuit 3 12 3H 12D  Using hand size of 5, with 100000 trials, has a success percentage of 11.9
Offsuit 3 12 3H 12D  Using hand size of 10, with 100000 trials, has a success percentage of 31.9
Offsuit 3 13 3H 13D  Using hand size of 5, with 100000 trials, has a success percentage of 6.1
Offsuit 3 13 3H 13D  Using hand size of 10, with 100000 trials, has a success percentage of 17.3
Offsuit 4 5 4H 5D  Using hand size of 5, with 100000 trials, has a success percentage of 39.1
Offsuit 4 5 4H 5D  Using hand size of 10, with 100000 trials, has a success percentage of 79.1
Offsuit 4 6 4H 6D  Using hand size of 5, with 100000 trials, has a success percentage of 36.6
Offsuit 4 6 4H 6D  Using hand size of 10, 

Offsuit 10 11 10H 11D  Using hand size of 10, with 100000 trials, has a success percentage of 26.8
Offsuit 10 12 10H 12D  Using hand size of 5, with 100000 trials, has a success percentage of 5.4
Offsuit 10 12 10H 12D  Using hand size of 10, with 100000 trials, has a success percentage of 19.5
Offsuit 10 13 10H 13D  Using hand size of 5, with 100000 trials, has a success percentage of 2.8
Offsuit 10 13 10H 13D  Using hand size of 10, with 100000 trials, has a success percentage of 10.7
Offsuit 11 12 11H 12D  Using hand size of 5, with 100000 trials, has a success percentage of 4.1
Offsuit 11 12 11H 12D  Using hand size of 10, with 100000 trials, has a success percentage of 15.8
Offsuit 11 13 11H 13D  Using hand size of 5, with 100000 trials, has a success percentage of 2.2
Offsuit 11 13 11H 13D  Using hand size of 10, with 100000 trials, has a success percentage of 8.7
Offsuit 12 13 12H 13D  Using hand size of 5, with 100000 trials, has a success percentage of 1.5
Offsuit 12 13 12H 13D